# Assignment 6: Rafael Niebles

### Disclaimer! This notebook assumes that:
- Teapot is at `./assets/teapot.obj`
- Brick texture is at `./assets/brick.jpg`
- The provided **LoadObject** script is in the same directory as this notebook

In [1]:
import glm
import math
import moderngl
import pygame

from LoadObject import getObjectData
from PIL import Image

class GraphicsEngine:
    def __init__(self, width=800, height=600):
        # Initialize Pygame
        pygame.init()

        # Enable forward compat; request context
        # https://stackoverflow.com/questions/76151435/creating-a-context-utilizing-moderngl-for-pygame
        pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MAJOR_VERSION, 3)
        pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MINOR_VERSION, 3)
        pygame.display.gl_set_attribute(
            pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE)
        pygame.display.gl_set_attribute(
            pygame.GL_CONTEXT_FORWARD_COMPATIBLE_FLAG, True)

        # We can set member vars like this
        self.WIDTH, self.HEIGHT = width, height

        pygame.display.set_mode((self.WIDTH, self.HEIGHT),
                                pygame.OPENGL | pygame.DOUBLEBUF)
        pygame.display.set_caption("Assignment 6: Rotating Camera")

        # Create MGL context
        self.ctx = moderngl.create_context()
        self.ctx.enable(moderngl.DEPTH_TEST)  # Enable Z-buffer
        self.ctx.clear_color = (0.0, 0.0, 0.0)

        # Load assets
        try:
            shape_data, self.bounds = getObjectData(
                "./assets/teapot.obj", texture=True)

            texture_image = Image.open('./assets/brick.jpg').convert('RGB')
            
            self.texture = self.ctx.texture(
                texture_image.size, 3, texture_image.tobytes())

        except FileNotFoundError as e:
            print(f"Missing: {e.filename}")
            pygame.quit()
            quit()
        
        # Shader code
        self.vertex_shader = """
        #version 330

        in vec3 in_position;
        in vec2 in_texcoord_0;
        
        uniform mat4 view;
        uniform mat4 perspective;
        
        // Output to frag. shader
        out vec2 v_texcoord;
        
        void main() {
            // Transform the vertex position to screen coordinates
            gl_Position = perspective * view * vec4(in_position, 1.0);
        
            // Pass the texture coordinate to the fragment shader
            v_texcoord = in_texcoord_0;
        }
        """

        self.fragment_shader = """
        #version 330

        in vec2 v_texcoord;
        
        uniform sampler2D u_texture;
        
        // Final output color for the pixel
        out vec4 f_color;
        
        void main() {
            // Get color from texture using texcoord!
            f_color = texture(u_texture, v_texcoord);
        }
        """

        # Shader program
        self.program = self.ctx.program(
            vertex_shader=self.vertex_shader,
            fragment_shader=self.fragment_shader
        )

        # VBO
        vbo = self.ctx.buffer(shape_data)

        # VAO
        self.vao = self.ctx.vertex_array(
            self.program,
            [(vbo, '3f 2f', 'in_position', 'in_texcoord_0')]
        )

        # The camera will always look at the center of the object
        self.lookAtPoint = glm.vec3(self.bounds.center)
        self.up_vector = glm.vec3(0, 1, 0)

        # Calc. initial camera pos
        angle_rad = glm.radians(60)
        distance = 2 * self.bounds.radius

        # Calc. initial pos relative to origin
        initial_cam_pos_relative = glm.vec3(
            0,  # On YZ plane, so X is 0
            distance * math.cos(angle_rad),  # Y component
            distance * math.sin(angle_rad)  # Z component
        )

        # Add the object's center to get the final world position
        self.initial_cam_pos = initial_cam_pos_relative + self.lookAtPoint

        # Perspective matrix
        fov = 45.0
        aspect_ratio = self.WIDTH / self.HEIGHT
        near_plane = self.bounds.radius
        far_plane = 3 * self.bounds.radius

        perspective_matrix = glm.perspective(
            glm.radians(fov), aspect_ratio, near_plane, far_plane)

        # Write uniforms
        self.view_uniform = self.program['view']
        self.perspective_uniform = self.program['perspective']
        self.texture_uniform = self.program['u_texture']

        self.perspective_uniform.write(perspective_matrix)
        self.texture_uniform.value = 0  # Tell the shader to use texture unit 0

    def run(self):
        running = True
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

            time = pygame.time.get_ticks() / 1000.0

            # Orbit around world Y axis
            rotation_angle = time * 0.5 # We can modify rot. speed if we want by changing this

            # Y axis rotation matrix
            rotation_matrix = glm.rotate(
                glm.mat4(1.0), rotation_angle, glm.vec3(0, 1, 0))

            # Translate the camera's initial pos. to origin
            # Apply rotation
            # Translate back
            cam_pos_relative_to_center = self.initial_cam_pos - self.lookAtPoint
            rotated_cam_pos = glm.vec3(
                rotation_matrix * glm.vec4(cam_pos_relative_to_center, 1.0))
            current_eye_point = rotated_cam_pos + self.lookAtPoint

            # Recalculate the view matrix for every frame using the new camera position
            view_matrix = glm.lookAt(
                current_eye_point, self.lookAtPoint, self.up_vector)


            self.view_uniform.write(view_matrix)

            # Render
            self.ctx.clear()

            self.texture.use(0)

            self.vao.render()

            pygame.display.flip()

        pygame.quit()


if __name__ == '__main__':
    app = GraphicsEngine()
    app.run()


/Users/rafa/Developer/GRAPHICS/.cgenv/lib/python3.13/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.13.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
Vertex position Count:  530 (5.929688, 4.125, 0.0)
Vertex normal Count:  530 (-0.966742, -0.255752, 0.0)
Vertex texture coordinate Count:  800 (2.0, 2.0, 0.0)
Face count:  992.0
texture exists
